In [1]:
from imgtools.io import read_dicom_series
from readii.loaders import loadSegmentation
from readii.image_processing import *

In [5]:
baseImage = read_dicom_series("../tests/NSCLC_Radiogenomics/R01-001/09-06-1990-NA-CT_CHEST_ABD_PELVIS_WITH_CON-98785/3.000000-THORAX_1.0_B45f-95741")

segImages = loadSegmentation("../tests/NSCLC_Radiogenomics/R01-001/09-06-1990-NA-CT_CHEST_ABD_PELVIS_WITH_CON-98785/1000.000000-3D_Slicer_segmentation_result-67652/1-1.dcm",
                            modality = 'SEG')
roiImage = segImages['Heart']
flattenedROIImage = flattenImage(roiImage)
baseROI = alignImages(baseImage, flattenedROIImage)

roiLabel = getROIVoxelLabel(baseROI)

randomSeed = 10

In [21]:
# convert CT to array
arrBaseImage = sitk.GetArrayFromImage(baseImage)

# convert baseROI to array
arrBaseROI = sitk.GetArrayFromImage(baseROI)

# make binary mask out of ROI (NaN for background, 1 for ROI)
binBaseROI = np.where(arrBaseROI > 0, 1, np.NaN)

# multiply the mask by the CT to get just ROI voxels
roiMaskedBaseImage = arrBaseImage * binBaseROI

In [28]:
maxROIVoxelVal = int(np.nanmax(roiMaskedBaseImage))
minROIVoxelVal = int(np.nanmin(roiMaskedBaseImage))

In [30]:
arrNewBaseImage = arrBaseImage.copy()

In [ ]:
# get count of non NaN values
# generate that many random values using max and min
# somehow get them into the correct shape of the ROI?

# or get size of ROI bounds
# make random array that size and multiply